<a href="https://colab.research.google.com/github/IkeLyons/HW2_DecisionTrees/blob/main/DecisionTrees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1

The first step is to import the same data used in homework 1 (note they are commented out so that when I run all they dont run every time, if you are running for the first time you may need to uncomment all these first lines)

In [11]:
! #pip install kaggle

In [12]:
! #mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [13]:
!#cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [14]:
! #kaggle datasets download tejashvi14/engineering-placements-prediction

  0% 0.00/10.0k [00:00<?, ?B/s]
100% 10.0k/10.0k [00:00<00:00, 25.1MB/s]


In [15]:
! #unzip engineering-placements-prediction.zip

Archive:  engineering-placements-prediction.zip
  inflating: collegePlace.csv        


In [16]:
import pandas as pd
dataset = pd.read_csv('collegePlace.csv', encoding='utf-8')

Now do the same preprocessing that we did in homework 1

In [17]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
dataset.Stream = encoder.fit_transform(dataset.Stream)
dataset.Gender = encoder.fit_transform(dataset.Gender)
# see if the label encoding worked
dataset.head()

,Age,Gender,Stream,Internships,CGPA,Hostel,HistoryOfBacklogs,PlacedOrNot
0,22,1,3,1,8,1,1,1
1,21,0,1,0,7,1,1,1
2,22,0,4,1,6,0,0,1
3,21,1,4,0,8,0,1,1
4,22,1,5,0,8,1,0,1


In [18]:
X = dataset.drop('PlacedOrNot', 1)
y = dataset['PlacedOrNot']

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()